In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# NWS API: Fetch temperature and precipitation
nws_headers = {"User-Agent": "WeatherAQIApp/1.0 #put_your_email_id_here#"}
lat, lon = 36.1540, -95.9928  # Tulsa, OK
nws_url = f"https://api.weather.gov/points/{lat},{lon}"

# AirNow API: Fetch AQI
airnow_api_key = # Replace with your AirNow API key
zip_code = "74101"  # Tulsa ZIP code
airnow_url = f"http://www.airnowapi.org/aq/forecast/zipCode/?format=application/json&zipCode={zip_code}&API_KEY={airnow_api_key}"

try:
    # Step 1: Get NWS forecast data
    nws_response = requests.get(nws_url, headers=nws_headers)
    nws_response.raise_for_status()
    nws_data = nws_response.json()

    # Get hourly forecast URL
    forecast_url = nws_data["properties"]["forecastHourly"]
    forecast_response = requests.get(forecast_url, headers=nws_headers)
    forecast_response.raise_for_status()
    forecast_data = forecast_response.json()

    # Extract temperature and precipitation for next 24 hours
    periods = forecast_data["properties"]["periods"][:24]  # First 24 hours
    weather_data = []
    for period in periods:
        weather_data.append({
            "Time": period["startTime"],
            "Temperature (°F)": period["temperature"],
            "Precipitation (%)": period.get("probabilityOfPrecipitation", {}).get("value", 0)
        })

    # Step 2: Get AirNow AQI data
    airnow_response = requests.get(airnow_url)
    airnow_response.raise_for_status()
    airnow_data = airnow_response.json()

    # Extract AQI for today (first available forecast)
    aqi_value = airnow_data[0]["AQI"] if airnow_data else "N/A"

    # Step 3: Combine data
    # Since AQI is daily, apply the same AQI value to all hourly entries
    for entry in weather_data:
        entry["AQI"] = aqi_value

    # Step 4: Create table with pandas
    df = pd.DataFrame(weather_data)
    df["Time"] = pd.to_datetime(df["Time"]).dt.strftime("%Y-%m-%d %H:%M")  # Format time
    print("\nWeather and AQI Table for Tulsa, OK:")
    print(df)

    # Step 5: Save to CSV
    df.to_csv("weather_aqi_tulsa.csv", index=False)
    print("\nData saved to 'weather_aqi_tulsa.csv'. Download from the Files tab.")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error: {http_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Error: {req_err}")
except Exception as e:
    print(f"Unexpected error: {e}")

In [ ]:
import requests
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from datetime import datetime

# Patient table (simulated)
patients = [
    {
        "id": 1,
        "name": "John Doe",
        "lat": 36.1540,
        "lon": -95.9928,
        "zip_code": "74101",
        "condition": "Asthma",
        "temp_threshold": 60,  # °F
        "aqi_threshold": 50,
        "email": ""  # Replace with patient's email
    },
    {
        "id": 2,
        "name": "Jane Smith",
        "lat": 36.1540,
        "lon": -95.9928,
        "zip_code": "74101",
        "condition": "COPD",
        "temp_threshold": None,  # No temp threshold
        "aqi_threshold": 50,
        "email": ""  # Replace with patient's email
    }
]

# API configurations
nws_headers = {"User-Agent": "PatientWeatherApp/1.0 #put_your_email_id_here#"}
airnow_api_key = # Replace with your AirNow API key
gmail_user = # Replace with your Gmail
gmail_password = # Replace with Gmail App Password (not regular password)

# Function to send email
def send_email(to_email, subject, body):
    try:
        msg = MIMEText(body)
        msg["Subject"] = subject
        msg["From"] = gmail_user
        msg["To"] = to_email

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(gmail_user, gmail_password)
            server.sendmail(gmail_user, to_email, msg.as_string())
        return True
    except Exception as e:
        print(f"Failed to send email to {to_email}: {e}")
        return False

# Initialize alert table
alerts = []

# Process each patient
for patient in patients:
    try:
        # Step 1: Fetch NWS weather data
        nws_url = f"https://api.weather.gov/points/{patient['lat']},{patient['lon']}"
        nws_response = requests.get(nws_url, headers=nws_headers)
        nws_response.raise_for_status()
        nws_data = nws_response.json()

        # Get hourly forecast
        forecast_url = nws_data["properties"]["forecastHourly"]
        forecast_response = requests.get(forecast_url, headers=nws_headers)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        # Get current hour's data (at 8:00 am correspongin to [20] value in list)
        current_period = forecast_data["properties"]["periods"][1]
        current_temp = current_period["temperature"]
        current_time = current_period["startTime"]

        # Step 2: Fetch AirNow AQI data
        airnow_url = f"http://www.airnowapi.org/aq/forecast/zipCode/?format=application/json&zipCode={patient['zip_code']}&API_KEY={airnow_api_key}"
        airnow_response = requests.get(airnow_url)
        airnow_response.raise_for_status()
        airnow_data = airnow_response.json()
        current_aqi = airnow_data[0]["AQI"] if airnow_data else "N/A"

        # Step 3: Check critical conditions
        alert_triggered = False
        alert_message = ""

        if patient["condition"] == "Asthma":
            if (patient["temp_threshold"] is not None and current_temp < patient["temp_threshold"] and
                current_aqi != "N/A" and current_aqi > patient["aqi_threshold"]):
                alert_triggered = True
                alert_message = (f"Warning: Conditions in Tulsa, OK are risky for your asthma. "
                                f"Temperature is {current_temp}°F (below {patient['temp_threshold']}°F) "
                                f"and AQI is {current_aqi} (above {patient['aqi_threshold']}). "
                                f"Consider staying indoors.")
        elif patient["condition"] == "COPD":
            if current_aqi != "N/A" and current_aqi > patient["aqi_threshold"]:
                alert_triggered = True
                alert_message = (f"Warning: Air quality in Tulsa, OK is risky for your COPD. "
                                f"AQI is {current_aqi} (above {patient['aqi_threshold']}). "
                                f"Consider staying indoors.")

        # Step 4: Generate and send alert
        if alert_triggered:
            alerts.append({
                "Patient ID": patient["id"],
                "Name": patient["name"],
                "Time": pd.to_datetime(current_time).strftime("%Y-%m-%d %H:%M"),
                "Temperature (°F)": current_temp,
                "AQI": current_aqi,
                "Alert Message": alert_message
            })

            # Send email alert
            if patient["email"]:
                subject = f"Weather Alert for {patient['name']}"
                sent = send_email(patient["email"], subject, alert_message)
                print(f"Alert for {patient['name']}: {alert_message}")
                print(f"Email sent to {patient['email']}: {sent}")
            else:
                print(f"Alert for {patient['name']}: {alert_message} (No email provided)")

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error for {patient['name']}: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error for {patient['name']}: {req_err}")
    except Exception as e:
        print(f"Unexpected error for {patient['name']}: {e}")

# Step 5: Create and save alert table
if alerts:
    df_alerts = pd.DataFrame(alerts)
    print("\nAlert Table:")
    print(df_alerts)
    df_alerts.to_csv("patient_weather_alerts.csv", index=False)
    print("\nAlerts saved to 'patient_weather_alerts.csv'. Download from the Files tab.")
else:
    print("\nNo alerts triggered for any patients.")